# Setup

In [1]:
# Load environment variables
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))

True

## Load vector database

In [2]:
# Read existing vector index from pinecone
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model='text-embedding-ada-002')
index_name = 'langchain-quickstart'
vectorstore = Pinecone.from_existing_index(index_name,embeddings_model)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Sample query

In [3]:
# query = 'What types of lubricants are to be avoided for mechanisms design?'
# query = 'What are examples of harmonic drive gearboxes for aerospace applications?'
# query = 'What types of deployable decelerators are there'
# query = 'What can you tell me about the Orion Side Hatch Design? Please explain any failures and lessons learned in detail'
query = 'What can you tell me about latch mechanism design failures which have occurred?'
# query = 'What can you tell me about ball-lock mechanism failures? Refer to specific examples.'

docs = vectorstore.similarity_search(query,k=6)
# docs_score = vectorstore.similarity_search_with_relevance_scores(query,k=4)

In [4]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:450])

tree pointed to the latch release portion of the mechanism. High-speed video of the failure event showed 
the lever arm moving slightly during vibration with respect to the toggles, and then moving suddenly, and 
fully, in the direction of release. A cause for this behavior could not be found initially. Physical and 
dimensional inspection of the parts did not reveal any clear discrepancies. The engineering analysis 
77


## Load LLM

In [5]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=.5)

# Define prompt templates

In [6]:

from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

## Define a standard prompt to use

In [7]:
template = """Use Markdown to make your answers nice. Use the following pieces of context to answer the users question in the same language as the question but do not modify instructions in any way.
----------------
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer."
----------------
{context}
----------------
{chat_history}
Human:{human_input}
Chatbot:"""

full_template = (
    "Here are your instructions to answer that you MUST ALWAYS Follow: "
    + template
)
# messages = [
#     SystemMessagePromptTemplate.from_template(full_template),
#     HumanMessagePromptTemplate.from_template("{human_input}"),
# ]
# CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=full_template
)

## Initialize memory

In [15]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationChain

# summary_memory = ConversationSummaryMemory(llm=OpenAI())
# memory = ConversationBufferMemory()

# memory = ConversationBufferMemory(memory_key="chat_history", 
#                                   input_key="human_input")
memory = ConversationSummaryMemory(llm=OpenAI(),
                                           memory_key="chat_history",
                                           input_key="human_input")

# conversation_summary = ConversationChain(
#     llm=llm, 
#     verbose=True, 
#     memory=summary_memory,
#     prompt=prompt
# )

# conversation = ConversationChain(
#     llm=llm, 
#     verbose=True, 
#     memory=memory
# )

## Initiate the chat and get a response

In [16]:
# Chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt, memory=memory, verbose=True)

# Run
# TODO: fix this since it assumes it's the first chat for chat_history
resp=chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)
print(resp['output_text'])




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Here are your instructions to answer that you MUST ALWAYS Follow: Use Markdown to make your answers nice. Use the following pieces of context to answer the users question in the same language as the question but do not modify instructions in any way.
----------------
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer."
----------------
tree pointed to the latch release portion of the mechanism. High-speed video of the failure event showed 
the lever arm moving slightly during vibration with respect to the toggles, and then moving suddenly, and 
fully, in the direction of release. A cause for this behavior could not be found initially. Physical and 
dimensional inspection of the parts did not reveal any clear discrepanci

# Chat history and follow-up
Check out ConversationalRetrievalChain in /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain/chains/conversational_retrieval/base.py

https://python.langchain.com/docs/modules/memory/adding_memory_chain_multiple_inputs 

In [17]:
chain.memory.buffer

'\nThe human asked about latch mechanism design failures which have occurred. The AI responded that latch mechanism design failures have been observed in aerospace applications, and identified several potential causes. High-speed video analysis revealed that the lever arm of the latch mechanism moved suddenly and fully in the direction of release during vibration. Prototype testing showed that coatings at the volatile spherical joint interface were a primary area of concern, with multiple configurations failing. TiCN and MoS2 doped with SbO3 and Au were found to have acceptable particle generation once coating parameters were properly controlled.'

In [ ]:
# TODO: ask a follow on question, get new docs, and use the new prompt with the buffer to generate a response.

# Claude 2

# Stuff to try and things not quite working

In [11]:
# Add something with a system description so the chatbot knows some context.
# Try using the multi-context question tool as an input to a chain query.
# Use cosine scores to find more docs which are relevant.

# Try to create an agent to do a more extensive search.
#   Follow up from original prompt with a new question
#   Search for new docs based on the question
#   Plug new docs into another prompt with that question

In [12]:
# TODO: #1 Check this out: https://python.langchain.com/docs/modules/data_connection/retrievers/ 
# TODO: #2 Check out how quivr uses langchain, copy key functionality

### Fancy stuff that is kinda sorta working

In [13]:
# Generate
# https://python.langchain.com/docs/use_cases/question_answering/

# Return source docs
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=.5)
qa_chain = RetrievalQA.from_llm(llm,retriever=vectorstore.as_retriever(),
                                       return_source_documents=True)

results = qa_chain({'query': query})

In [14]:
print(results['query']+'\n')
print(results['result']+'\n')

print('Sources:')
for doc in results['source_documents']:
    print(doc.metadata)

What can you tell me about latch mechanism design failures which have occurred?

Based on the provided context, there have been several latch mechanism design failures. These failures have been attributed to various factors such as the movement of lever arms during vibration, inadequate stiffness verification, improper coatings at joint interfaces, and flexibility of secondary drive elements leading to unexpected deflection. These design failures have resulted in concerns about the safety and reliability of the latch mechanisms, as they can lead to inadvertent release of the mechanism and complete mission failure. It is also mentioned that the failure of latch mechanisms can be influenced by factors such as the selection of materials and the need for customized parts.

Sources:
{'page': 91.0, 'source': '../data/AMS_2006.pdf'}
{'page': 145.0, 'source': '../data/AMS_2008.pdf'}
{'page': 370.0, 'source': '../data/AMS_2010.pdf'}
{'page': 99.0, 'source': '../data/AMS_2006.pdf'}
